In [1]:
import pandas as pd

# increase the output column width
pd.set_option('display.max_colwidth', 200)

import numpy as np
import re

# for NLP related tasks
#!pip install spacy

import spacy
# nlp=spacy.load('en_core_web_sm',disable=["tagger", "parser","ner"])

In [2]:
#!python -m spacy download en_core_web_sm

In [3]:
import en_core_web_sm

nlp = en_core_web_sm.load()

<font size=5>**Pipeline**</font>
1. Loading and Exploring Data
2. Text Cleaning
3. Data Preparation
    1. Label Encoding
    2. Split Data
    3. Feature Engineering using TF-IDF
4. Model Building
    1. Naive Bayes
    2. Logistic Regression
    3. Model Building Summary
5. Final Sentiment Analysis Pipeline

# Loading and Exploring Data

In [4]:
# mounting the drive
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# read CSV file
df = pd.read_csv('Tweets.csv')

#shape of the dataframe
print('Shape=>',df.shape)

# print first 5 rows
df.head()

Shape=> (14640, 15)


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials to the experience... tacky.,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I need to take another trip!,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse",NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing about it,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [6]:
# Some sample tweets
df['text'].sample(5)

8252                                                                                                                 @JetBlue thanks JB!! This is why I 💙 you!
4781    @SouthwestAir thank you for Cancelled Flighting my flight, last minute of course, to #GIS2015. I'm now missing one full day of my trip thanks to this!
4230                                                                @united  flight 441 on 2/4 was delayed over three hours because the pilot never showed up!
2326                                                                                                             @united this team added 30 mins on turnaround
9428                               @USAirways going to miss my connection due to another tarmac delay - on US5268 then 4050 - need a flight home today please!
Name: text, dtype: object

In [7]:
# class distribution
df['airline_sentiment'].value_counts()

airline_sentiment
negative    9178
neutral     3099
positive    2363
Name: count, dtype: int64

In [8]:
# class distribution in percentage
df['airline_sentiment'].value_counts(normalize = True)*100

airline_sentiment
negative    62.691257
neutral     21.168033
positive    16.140710
Name: proportion, dtype: float64

# Text Cleaning

In [9]:
#define a function for text cleaning
def text_cleaner(text):
  
  #remove user mentions
  text = re.sub(r'@[A-Za-z0-9]+','',text)           
  
  #remove hashtags
  #text = re.sub(r'#[A-Za-z0-9]+','',text)         
  
  #remove links
  text = re.sub(r'http\S+', '', text)  

  #convering text to lower case
  text = text.lower()

  # fetch only words
  text = re.sub("[^a-z]+", " ", text)

  # removing extra spaces
  text=re.sub("[\s]+"," ",text)
  
  # creating doc object
  doc=nlp(text)

  # remove stopwords and lemmatize the text
  tokens=[token.lemma_ for token in doc if(token.is_stop==False)]
  
  #join tokens by space
  return " ".join(tokens)

In [10]:
# perform text cleaning
df['clean_text']= df['text'].apply(text_cleaner)

In [11]:
# save cleaned text and labels to a variable
text   = df['clean_text'].values
labels = df['airline_sentiment'].values

In [12]:
# Sample cleaned text
text[:10]

array(['  say', '  plus ve add commercial experience tacky',
       '  didn t today mean need trip',
       '  s aggressive blast obnoxious entertainment guest face amp little recourse',
       '  s big bad thing',
       '  seriously pay flight seat didn t playing s bad thing fly va',
       '  yes nearly time fly vx ear worm win t away',
       '  miss prime opportunity man hat parody', '  didn t d',
       '  amazing arrive hour early good'], dtype=object)

In [13]:
# Sample labels
labels[:10]

array(['neutral', 'positive', 'neutral', 'negative', 'negative',
       'negative', 'positive', 'neutral', 'positive', 'positive'],
      dtype=object)

# Data Preparation

## Label Encoding

In [14]:
#importing label encoder
from sklearn.preprocessing import LabelEncoder

#define label encoder
le = LabelEncoder()

#fit and transform target strings to a numbers
labels = le.fit_transform(labels)

In [15]:
# Sample labels
labels[:10]

array([1, 2, 1, 0, 0, 0, 2, 1, 2, 2])

In [16]:
# Meaning of each label
le.inverse_transform([0,1,2])

array(['negative', 'neutral', 'positive'], dtype=object)

## Split Data

In [17]:
from sklearn.model_selection import train_test_split

# Splitting into train and validation set
x_train,x_val,y_train,y_val=train_test_split(text, labels,stratify=labels, test_size=0.2, random_state=0,shuffle=True)

In [18]:
print('x_train:',x_train.shape,'y_train:',y_train.shape)
print('x_val:',x_val.shape,'y_val:',y_val.shape)

x_train: (11712,) y_train: (11712,)
x_val: (2928,) y_val: (2928,)


## Feature Engineering using TF-IDF

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
# initialize TFIDF
word_vectorizer = TfidfVectorizer(max_features=1000)

In [21]:
# Fitting Vectorizer on Train set
word_vectorizer.fit(x_train)

TfidfVectorizer(max_features=1000)

In [22]:
# create TF-IDF vectors for Train Set
train_word_features = word_vectorizer.transform(x_train)
train_word_features

<11712x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 70569 stored elements in Compressed Sparse Row format>

In [23]:
# create TF-IDF vectors for Validation Set
val_word_features = word_vectorizer.transform(x_val)
val_word_features

<2928x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 17861 stored elements in Compressed Sparse Row format>

# Model Building

## Naive Bayes

In [24]:
# Importing for modeling
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

In [25]:
# Training model
nb_model=MultinomialNB().fit(train_word_features,y_train)
nb_model

MultinomialNB()

In [26]:
# Make predictions for train set
train_pred_nb=nb_model.predict(train_word_features)

In [27]:
train_pred_nb

array([0, 0, 0, ..., 0, 0, 0])

In [28]:
# Evaluating on Training Set
print("F1-score on Train Set:",f1_score(y_train,train_pred_nb,average="weighted"))

F1-score on Train Set: 0.728650883478306


In [29]:
# Make predictions for validation set
val_pred_nb=nb_model.predict(val_word_features)

# Evaluating on Validation Set
print("F1-score on Validation Set:",f1_score(y_val,val_pred_nb,average="weighted"))

F1-score on Validation Set: 0.681555775964176


## Logistic Regression

In [30]:
from sklearn.linear_model import LogisticRegression

In [31]:
# Training model
lr_model=LogisticRegression().fit(train_word_features,y_train)
lr_model

C:\Users\Phoenix\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [32]:
# Make predictions for train set
train_pred_lr=lr_model.predict(train_word_features)
train_pred_nb

array([0, 0, 0, ..., 0, 0, 0])

In [33]:
# Evaluating on Training Set
print("F1-score on Train Set:",f1_score(y_train,train_pred_lr,average="weighted"))

F1-score on Train Set: 0.8106951801857508


In [34]:
# Make predictions for validation set
val_pred_lr=lr_model.predict(val_word_features)

# Evaluating on Validation Set
print("F1-score on Validation Set:",f1_score(y_val,val_pred_lr,average="weighted"))

F1-score on Validation Set: 0.758370332152591


## Model Building Summary
|        Model        | Train Set | Validation Set |
|:-------------------:|:---------:|:--------------:|
|     Naive Bayes     |   0.7274  |     0.6791     |
| Logistic Regression |   0.8089  |     0.7598     |

It is evident from the results that Logistic Regression performs better than Naive Bayes on this dataset.

# Final Sentiment Analysis Pipeline

In [35]:
def sentiment_analyzer(tweet):
  # Cleaning Tweet
  cleaned_tweet=text_cleaner(tweet)

  # Feature Engineering
  tweet_vector=word_vectorizer.transform([cleaned_tweet])

  # Predicting Sentiment
  label=lr_model.predict(tweet_vector)

  return le.inverse_transform(np.array(label))

<font size=4>**Sample Tweet:**</font>
<p>@USAirways flt 419. 2+ hrs Late Flight, baggage + 1 more hr. Now I see they delivered my suitcase wet inside &amp; out. #NotHappy</p>

In [36]:
sentiment_analyzer("@USAirways flt 419. 2+ hrs Late Flight, baggage + 1 more hr. Now I see they delivered my suitcase wet inside &amp; out. #NotHappy")

array(['negative'], dtype=object)